# IMPORT_DATA _FROM_LOCAL_to_MYSQL_USING_JDBC

In [1]:
import mysql.connector as connector
from mysql.connector import errorcode


## Establish connection with MySQL Database

In [4]:
try:
    con = connector.connect(host='localhost',\
                            port ='3306',\
                            user ='root',password='Root123@',\
                            use_pure=True,\
                            allow_local_infile=True)
    cur = con.cursor(buffered=True)
    print("Connected  successfully.")
except connector.Error as e:
    print("Failed creating connection: {}".format(e))

Connected  successfully.


## Creating the Database "big_data"

In [ ]:
DB_NAME = 'big-data'

try:
    cur.execute(
        "CREATE DATABASE {" "} ".format(DB_NAME))
    print("Database {} created successfully.".format(DB_NAME))
except connector.Error as e:
    print("Failed creating database: {}".format(e))

try:
    cur.execute("USE {}".format(DB_NAME))
except connector.Error as e:
    print("Database {} does not exists.".format(DB_NAME))
    

## Creating the table schemas for our data

In [4]:
T = ['employees', 'departments', 'salaries', 'dept_manager', 'titles', 'dept_emp']

for tbl in T:
    try:
        cur.execute("DROP TABLE IF EXISTS {}".format(tbl))
    except connector.Error as e:
        print('e')

In [5]:

TABLES = {}

TABLES['employees'] = ("CREATE TABLE employees (emp_no VARCHAR(20),emp_title_id VARCHAR(20),birth_date VARCHAR(20),\
                                                first_name VARCHAR(20),last_name VARCHAR(20),sex VARCHAR(20),\
                                                hire_date VARCHAR(20),no_of_projects VARCHAR(20),\
                                                Last_performance_rating VARCHAR(20),left_ VARCHAR(20),last_date VARCHAR(20))")

TABLES['departments'] = ("CREATE TABLE departments (dept_no VARCHAR(20),dept_name VARCHAR(20))")

TABLES['salaries'] = ("CREATE TABLE salaries (emp_no VARCHAR(20),salary VARCHAR(20))")

TABLES['dept_manager'] = ("CREATE TABLE dept_manager (dept_no VARCHAR(20), emp_no VARCHAR(20))")

TABLES['titles'] = ("CREATE TABLE titles (title_id VARCHAR(20),title VARCHAR(20))")

TABLES['dept_emp'] = ("CREATE TABLE dept_emp (emp_no VARCHAR(20),dept_no VARCHAR(20))")


for table_name in TABLES:
    schema = TABLES[table_name]
    try:
        cur.execute(schema)
        print("Table {" "} created. ".format(table_name))
    except connector.Error as e:
        print("Failed to create TABLE : {}".format(e))
    else:
        print("OK")


Table employees created. 
OK
Table departments created. 
OK
Table salaries created. 
OK
Table dept_manager created. 
OK
Table titles created. 
OK
Table dept_emp created. 
OK


# Adding the primary key and foreign key to the tables

In [6]:

TABLE = {}

TABLES['titles'] = ("ALTER TABLE titles ADD PRIMARY KEY(title_id)")
TABLES['employees'] = ("ALTER TABLE employees ADD PRIMARY KEY(emp_no)")
TABLES['employees'] = ("ALTER TABLE employees ADD FOREIGN KEY(emp_title_id) REFERENCES titles(title_id)")
TABLES['salaries'] = ("ALTER TABLE salaries ADD FOREIGN KEY(emp_no) REFERENCES employees(emp_no)")
TABLES['departments'] = ("ALTER TABLE departments ADD PRIMARY KEY(dept_no)")
TABLES['dept_emp'] = ("ALTER TABLE dept_emp ADD FOREIGN KEY(emp_no) REFERENCES employees(emp_no)")
TABLES['dept_emp'] = ("ALTER TABLE dept_emp ADD FOREIGN KEY(dept_no) REFERENCES departments(dept_no)")
TABLES['dept_manager'] = ("ALTER TABLE dept_manager ADD FOREIGN KEY(emp_no) REFERENCES employees(emp_no)")
TABLES['dept_manager'] = ("ALTER TABLE dept_manager ADD FOREIGN KEY(dept_no) REFERENCES departments(dept_no)")

for table_name in TABLE:
    schema = TABLE[table_name]
    try:
        cur.execute(schema)
        print("keys of {" "} created. ".format(table_name), end='')
    except connector.Error as e:
        print("Failed to create TABLE : {}".format(e))
    else:
        print("OK")

## Loading the data from local-system to Mysql

In [7]:
load_data ={}

load_data['employees'] = ("LOAD DATA LOCAL INFILE '/root/employee-data/Data/employees.csv' \
                                            INTO TABLE employees FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' \
                                            IGNORE 1 rows")

load_data['departments'] = ("LOAD DATA LOCAL INPATH '/root/employee-data/Data/departments.csv' \
                                            INTO TABLE departments FIELDS TERMINATED BY ';' LINES TERMINATED BY '\n' \
                                            IGNORE 1 rows")

load_data['salaries'] = ("LOAD DATA LOCAL INFILE '/root/employee-data/Data/salaries.csv' \
                                    INTO TABLE salaries FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' \
                                    IGNORE 1 rows")

load_data['dept_manager'] = ("LOAD DATA LOCAL INFILE '/root/employee-data/Data/dept_manager.csv' \
                                                INTO TABLE dept_manager FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' \
                                                IGNORE 1 rows")

load_data['titles'] = ("LOAD DATA LOCAL INFILE '/root/employee-data/Data/titles.csv' \
                                INTO TABLE titles FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' \
                                IGNORE 1 rows")

load_data['dept_emp'] = ("LOAD DATA LOCAL INFILE '/root/employee-data/Data/dept_emp.csv' \
                                        INTO TABLE dept_emp FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' \
                                        IGNORE 1 rows")



In [8]:
try:
    cur.execute("USE {}".format(DB_NAME))
except connector.Error as e:
    print("Database {} does not exists.".format(DB_NAME))

for table_name in load_data:
    table_description = load_data[table_name]
    try:
        cur.execute(table_description)
        print("Load data into {} table: ".format(table_name))
    except connector.Error as e:
        print("Failed to create TABLE : {}".format(e))
    

Load data into employees table: 
Load data into departments table: 
Load data into salaries table: 
Load data into dept_manager table: 
Load data into titles table: 
Load data into dept_emp table: 


## Check if the data was loaded into the table successfully

In [9]:
tables = ['employees', 'departments', 'salaries', 'dept_manager', 'titles', 'dept_emp']

try:
    cur.execute("USE {}".format(DB_NAME))
except connector.Error as e:
    print("Database {} does not exists.".format(DB_NAME))

for table_name in tables:
    try:
        cur.execute("select count(*) from {} ".format(table_name))
        total=cur.fetchall()
        print("Total number of ROWS in {} Table: {} ".format(table_name,total))
    except connector.Error as e:
        print("Table {} does not exists : {}".format(tables))

Total number of ROWS in employees Table: [(300024,)] 
Total number of ROWS in departments Table: [(9,)] 
Total number of ROWS in salaries Table: [(300024,)] 
Total number of ROWS in dept_manager Table: [(24,)] 
Total number of ROWS in titles Table: [(7,)] 
Total number of ROWS in dept_emp Table: [(331603,)] 


In [10]:
cur.execute("select * from titles limit 5")
for detail in cur:
    print(detail)

('s0001', 'Staff')
('s0002', 'Senior Staff')
('e0001', 'Assistant Engineer')
('e0002', 'Engineer')
('e0003', 'Senior Engineer')


In [11]:
# cur.close()
# con.close()